# Representative library virus titers from 042922 virus batch, unmutated integrated virus titers from 120720, and unmutated virus transfected virus titers from 101220

This was the very first library rescue by Caleb Carr that is being used as a representative library titer and representative virus titers for unmutated GPC carried out by Kate Crawford while designing DMS libraries.

In [ ]:
# Imports
import os
import warnings
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Plotting colors
tol_muted_adjusted = [
    "#000000",
    "#CC6677", 
    "#1f78b4", 
    "#DDCC77", 
    "#117733", 
    "#882255", 
    "#88CCEE",
    "#44AA99", 
    "#999933", 
    "#AA4499", 
    "#EE7733",
    "#CC3311",
    "#DDDDDD",
]

# Seaborn style settings
sns.set(rc={
    "figure.dpi":300, 
    "savefig.dpi":300,
    "svg.fonttype":"none",
})
sns.set_style("ticks")
sns.set_palette(tol_muted_adjusted)

# Suppress warnings
warnings.simplefilter("ignore")

In [ ]:
# Read data
library_data = pd.read_excel("data/042922_titer_calculations.xlsx")
unmutated_int_data = pd.read_csv("data/201207_titer_calculations.csv")
unmutated_trans_data = pd.read_csv("data/201012_titer_calculations.csv")
# Set output directory
out_dir = "figures/"

In [ ]:
# Re-group data
library_data = library_data[["Infection_sample", "Titers_(TU/mL)"]]
library_data["Library"] = library_data["Infection_sample"].str.contains("_A_")
library_data["Library"] = library_data["Library"].map({True: "library A variants", False: "library B variants"})
library_data["Infection_sample"] = library_data["Infection_sample"].str.replace("Library_A_", "")
library_data["Infection_sample"] = library_data["Infection_sample"].str.replace("Library_B_", "")
library_data["Infection_sample"] = library_data["Infection_sample"].str.replace("_1", "")
library_data["Infection_sample"] = library_data["Infection_sample"].str.replace("_2", "")
library_data["Date"] = "042922"

# Re-index to have increasing values
library_data = (
    library_data
    .reindex([6,7,2,3,0,1,4,5])
    .reset_index(drop=True)
)
# Set library column to mutated for no HPs 
# as a dummy value so it shows up in the legend
library_data.at[0, "Library"] = "unmutated"

# Add mock library column to other data
unmutated_int_data["Library"] = "unmutated"
unmutated_int_data["Date"] = "120720"
unmutated_trans_data["Library"] = "unmutated"
unmutated_trans_data["Date"] = "101220"

# Merge dataframes
merged_df = (
    pd.concat([
        library_data, 
        unmutated_int_data,
        unmutated_trans_data,
    ])
    .drop(columns=[
        "Percent_infected",
        "Number_of_cells_infected",
        "Dilution_factor",
    ])
    .reset_index(drop=True)
)

In [ ]:
# Plot data
fig, axes = plt.subplots(
    1, 
    3, 
    figsize=(5,2),
    # gridspec_kw={"width_ratios": [1, 0.5, 0.5]}
)

for index,date in enumerate(["042922", "120720", "101220"]):
    
    # Extract data for date
    data_frame = merged_df.loc[merged_df["Date"] == date]

    chart = sns.scatterplot(
        data=data_frame, 
        x="Infection_sample", 
        y="Titers_(TU/mL)", 
        alpha=0.8,
        hue="Library",
        palette={
            "library B variants" : "#000000", 
            "library A variants" : "#DDDDDD",
            "unmutated" : "#FFFFFF",
        },
        edgecolor="black",
        linewidth=0.5,
        s=40,
        ax=axes[index],
    )
    chart.set_yscale("log")
    chart.set_ylim(5, 50000000) # put at 0 to allow for lowest point to not be cut off
    yticks = [10, 1000, 100000, 10000000]
    chart.set_yticks(yticks)
    chart.set_yticklabels(labels=["$10^1$", "$10^3$", "$10^5$", "$10^7$"], fontsize=8)
    if index == 0:
        chart.set_title(
            f"from cells with\nintegrated library\nGPC variants\ntitrated on {date}",
            loc="left", 
            horizontalalignment="left", 
            fontsize=8,
            # weight="bold",
        )
        xticks = [0, 1, 2, 3]
        chart.set_xticks(xticks)
        # Re-do x-axis in illustator
        x_labels = ["No_Plasmids", "HPs_unconcentrated", "HPs_concentrated",  "HPs+VSVG"]
    if index == 1:
        chart.set_title(
            f"from cells with\nintegrated\nunmutated GPC\ntitrated on {date}",
            loc="left", 
            horizontalalignment="left", 
            fontsize=8,
            # weight="bold",
        )
        xticks = [0]
        chart.set_xticks(xticks)
        # Re-do x-axis in illustator
        x_labels = ["HPs_integrated"]
    if index == 2:
        chart.set_title(
            f"from cells with\ntransfected\nunmutated GPC\ntitrated on {date}",
            loc="left", 
            horizontalalignment="left", 
            fontsize=8,
            # weight="bold",
        )
        xticks = [0]
        chart.set_xticks(xticks)
        # Re-do x-axis in illustator
        x_labels = ["HPs_transfected"]
    chart.set_xticklabels(labels=x_labels, rotation=90, horizontalalignment="center", fontsize=8)
    chart.set_ylabel("transduction units per mL", fontsize=8)
    chart.set(xlabel=None)
    if index == 0:
        sns.move_legend(
            chart, 
            "upper left", 
            bbox_to_anchor=(3.5, 0.5),
            fontsize=8,
            markerscale=1,
            handletextpad=0.1,
            title="Lassa GPC",
            title_fontproperties = {
                "size" : 8, 
                "weight" : "bold",
            },
            frameon=False,
            borderaxespad=0.1,
        )
        # Add edges to legend markers to match scatter plot
        for ha in chart.legend_.legendHandles:
            ha.set_edgecolor("black")
            ha.set_linewidths(0.5)
            
    else:
        chart.get_legend().remove()

    # Only keep the first y-axis
    if index == 0:
        # Change all spines
        for axis in ["top", "bottom", "left", "right"]:
            chart.spines[axis].set_linewidth(1)
        chart.tick_params(axis="both", length=4, width=1)
    else:
        # Change all spines
        for axis in ["top", "bottom", "left", "right"]:
            chart.spines[axis].set_linewidth(1)
        # Remove y-axis
        chart.spines["left"].set_linewidth(0)
        chart.set_yticks([])
        chart.set_yticklabels([])
        chart.tick_params(axis="both", length=4, width=1)
        chart.set_ylabel("")
    
    chart.grid(False)
    sns.despine()

# Make output dir if doesn't exist
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    
# Common X and Y axis labels
fig.text(
    0.125, 
    1.2, 
    "representative pseudovirus titers", 
    ha="left", 
    rotation="horizontal", 
    fontsize=8, 
    weight="bold",
)

# Save fig
plt.savefig(out_dir + "representative_library_titers.svg")